<a href="https://colab.research.google.com/github/Anshuman-37/Summer_Dissertation/blob/main/Summer_Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prerequisites 

In [ ]:
# Upload the zip file and then unzip it
# unzipping the file in the directory 
!unzip /content/Raw\ Files.zip -d /content/
!unzip /content/Baseline.zip -d /content/

### Installing Libraries

In [114]:
# Install nibabel
!pip install nibabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing Files 

In [115]:
## importing libraries 
import os
import re
import nibabel as nib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,5)


### Code

### Reading the Data

##### Getting path of data

In [135]:
## Go in to the directories
## Extracting only important data using Regex
path = '/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/'
regex = re.compile(r'__MPR.*') ; fileRegex1 = re.compile(r'(WIP)*T13D'); mri_data_path = []; 

# Iterating over Directories 
for subdir, dirs, files in os.walk(path):
    for file in files:
        # Storing the path of Data
        if regex.search(file) == None:
            if fileRegex1.search(file): mri_data_path.append(os.path.join(subdir, file));
# Printing the path of Data
for i in Data_path: print(i); 

/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1277547 dicom/__WIP_MPRAGE_T13D_SENSE_20150430160250_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1320961 dicom/__WIP_MPRAGE_T13D_SENSE_20160115152740_301.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1335478 dicom/__WIP_MPRAGE_T13D_SENSE_20160407112645_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1289317 dicom/__WIP_MPRAGE_T13D_SENSE_20150710135121_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1291194 dicom/__WIP_MPRAGE_T13D_SENSE_20150724132211_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1327832 dicom/__WIP_MPRAGE_T13D_SENSE_20160222133236_301.nii


##### Loading data using nibabel

In [136]:
mri_images = []

# Loading data in the image vector
for i in mri_data_path: mri_images.append(nib.load(i)); 

# Printing the shape of images stored and its data type
for i in mri_images: print('Image shape ->',i.shape,end = '\t'); print('Image Data Type ->',i.get_data_dtype());

Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16


###### Discussions -

1. - The Data that is loaded right now is our X(Attribute) and Y should be our ASL Measured Data
2. - The nii files are a 3D tensor with some values ranging between 0-400k (IDK what it is but we need to ask her in the next meeting)


###### Things we do next -
1. - Try to normalize the Values of X first
2. - Understanding Y (ASL data) is next
3. - We write a model for using X to predict Y



###### Data Analysis 

In [137]:
## Reading the Data
img = mri_images[0]
a = np.array(img.dataobj)
## Use to check whether there is anything there or not
# print(a)
# Getting maximum values from the np array
a_max = np.amax(a); a_min = np.amin(a);
print(a_max); print(a_min);  # To ask where this maximum value comming from

397054.06677246094
0.0


In [138]:
# Just checking the time taken to iterate over
%timeit np.amin(a); 
%timeit np.amax(a); 

100 loops, best of 5: 9.76 ms per loop
100 loops, best of 5: 9.76 ms per loop


In [139]:
## Min max noramalization
a[:,:,:] = (a - a.min()) / (a.max() - a.min())

In [ ]:
# Normalized
a_max = np.amax(a); a_min = np.amin(a);
print(a_max); print(a_min);

# Unique counts 
unique, counts = np.unique(a, return_counts=True)
freq_counts = np.vstack(( unique, counts)).T
print(freq_counts)

## Uncomment to plot the graph
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.bar(unique,counts)
# # Most of the normalized values lies at less than <0.40 of maximum 
# # There is a lot of data imbalance  
# plt.show()

###### Discussions -

1. - There is a huge unique data imbalance most of the values after noramzalization are below 40% of maximum value.

###### Things we do next -
1. - Load Y
2. - Understanding Y ASL data
3. - Normalizing Y

###### Reading the Target Data (ASL)

In [141]:
# Getting the path for ASL files
asl_path = '/content/Baseline/'
asl_data_path = []

# Iterating over directories
for subdir, dirs, files in os.walk(asl_path):
    # Selecting the ASL file
    for file in files:
        if file == 'asldata.nii': asl_data_path.append(os.path.join(subdir, file));

# Printing the path of ASL images
for i in asl_data_path: print(i);


/content/Baseline/1277547 dicom/analysis/asldata.nii
/content/Baseline/1320961 dicom/analysis/asldata.nii
/content/Baseline/1289317 dicom/analysis/asldata.nii
/content/Baseline/1291194 dicom/analysis/asldata.nii
/content/Baseline/1259089 dicom/sub0001/asldata.nii
/content/Baseline/1327832 dicom/analysis/asldata.nii


In [142]:
asl_images = []

# Loading data in the image vector
for i in asl_data_path: asl_images.append(nib.load(i)); 

# Printing the shape of images stored and its data type
for i in asl_images: print('Image shape ->',i.shape,end = '\t'); print('Image Data Type ->',i.get_data_dtype());

Image shape -> (80, 80, 13, 60)	Image Data Type -> int16
Image shape -> (80, 80, 13, 60)	Image Data Type -> int16
Image shape -> (80, 80, 13, 60)	Image Data Type -> int16
Image shape -> (80, 80, 13, 60)	Image Data Type -> int16
Image shape -> (80, 80, 13, 60)	Image Data Type -> int16
Image shape -> (80, 80, 13, 60)	Image Data Type -> int16


##### EDA and Normalization of ASL DATA

In [143]:
## Reading the Data
img = asl_images[0]
a = np.array(img.dataobj)
## Use to check whether there is anything there or not
# print(a)
# Getting maximum values from the np array
a_max = np.amax(a); a_min = np.amin(a);
print(a_max); print(a_min); 

868295.3192138672
0.0


In [144]:
# Just checking the time taken to iterate over
%timeit np.amin(a); 
%timeit np.amax(a); 

100 loops, best of 5: 2.48 ms per loop
100 loops, best of 5: 2.41 ms per loop


In [145]:
## Min max noramalization
a[:,:,:] = (a - a.min()) / (a.max() - a.min())

In [ ]:
# Normalized
a_max = np.amax(a); a_min = np.amin(a);
print(a_max); print(a_min);

# Unique counts 
unique, counts = np.unique(a, return_counts=True)
freq_counts = np.vstack(( unique, counts)).T
print(freq_counts)


# # Uncomment to plot the graph
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.bar(unique,counts)
# # Most of the normalized values lies at less than <0.40 of maximum 
# # There is a lot of data imbalance  
# plt.show()

###### Discussions -

1. - The values of ASL are kind of similar after the min max normalization. This means we can train a model and get good results. 

###### Things we do next -
1. - Try to make the data a bit more better 
2. - Understanding X and Y even better
3. - Trying to make things a notch simpler